### Acceso a los datos de ventas

In [35]:
# Importación de librerías necesarias

import pandas as pd
import re

Carga el dataset de ventas. ¿Cuántas unidades de cualquier helado se han vendido en total a lo largo del histórico?

In [36]:
# Lee el archivo CSV
df_ventas = pd.read_excel('./ventas_helados.xlsx')

# Muestra las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
print(df_ventas.head())
print("")

# Calcular el total de unidades de helado vendidas
total_unidades_vendidas = df_ventas['ventas_unidades'].sum()
print("El total de unidades de helado vendidas en el histórico es:", total_unidades_vendidas)

Primeras filas del DataFrame:
       fecha  id_producto  ventas_unidades  precio
0 2019-01-07         4758             7953    4.82
1 2019-01-07           17             7052    1.50
2 2019-01-07         9134             6403    2.36
3 2019-01-07         8689             4266    2.47
4 2019-01-07          402             2932    2.89

El total de unidades de helado vendidas en el histórico es: 3848068


¿Cuál es el identificador del helado con más ventas en euros?

In [37]:
# Calcular las ventas totales por producto (en euros)
df_ventas['ventas_euros'] = df_ventas['ventas_unidades'] * df_ventas['precio']

# Agrupar por id_producto y sumar las ventas en euros
ventas_por_producto = df_ventas.groupby('id_producto')['ventas_euros'].sum()

# Ordenar las ventas por producto de mayor a menor
ventas_por_producto_ordenadas = ventas_por_producto.sort_values(ascending=False)

print("Ventas por producto ordenadas de mayor a menor:\n")
print(ventas_por_producto_ordenadas)

# Encontrar el identificador del producto con más ventas en euros
id_producto_mas_vendido = ventas_por_producto.idxmax()

print("\nEl identificador del helado con más ventas en euros es:", id_producto_mas_vendido)

Ventas por producto ordenadas de mayor a menor:

id_producto
4758    2247420.20
8689    1758488.86
9134    1619742.42
17      1491649.79
402      797522.39
4577     672285.39
2033     518685.75
240      478199.22
5684     166786.80
6525     146400.29
101      102972.81
3636      83036.58
5019      43555.95
999       37850.19
4468      37534.98
7118       9602.29
Name: ventas_euros, dtype: float64

El identificador del helado con más ventas en euros es: 4758


### Acceso a los datos de publicidad

Lee los datos de publicidad, y transformalos en una serie semanal con la suma de todos los medios.

In [38]:
# Cargar el dataset de inversión en publicidad
df_inversiones = pd.read_excel('./inversiones_helados.xlsx')
print('Dataset de inversión en publicidad\n')
print(df_inversiones)

Dataset de inversión en publicidad

         fecha  inversion_tv  inversion_radio  inversion_youtube  \
0   2019-01-07          6100                0               6100   
1   2019-01-14             0                0                  0   
2   2019-01-21          1120                0               1120   
3   2019-01-28             0                0                  0   
4   2019-02-04           880              880                880   
..         ...           ...              ...                ...   
99  2020-11-30          4880             4880               4880   
100 2020-12-07         17580            17580                  0   
101 2020-12-14         26520            26520              26520   
102 2020-12-21           860                0                860   
103 2020-12-28           960              960                960   

     inversion_facebook  
0                     0  
1                  6530  
2                  5610  
3                  1510  
4                

In [39]:
# Agrupar por semana (lunes a domingo) y sumar los valores para cada medio
serie_semanal = df_inversiones.groupby(pd.Grouper(key='fecha', freq='W-MON'))\
    .sum()\
    .sum(axis=1)

print("Serie semanal con la suma de inversión en todos los medios:")
print(serie_semanal)


Serie semanal con la suma de inversión en todos los medios:
fecha
2019-01-07    12200
2019-01-14     6530
2019-01-21     7850
2019-01-28     1510
2019-02-04     2640
              ...  
2020-11-30    14640
2020-12-07    35160
2020-12-14    79560
2020-12-21     5970
2020-12-28     7680
Freq: W-MON, Length: 104, dtype: int64


¿Cuál es el mes con más inversión total?

In [40]:
# Convertir los índices de la serie a formato de fecha
serie_semanal.index = pd.to_datetime(serie_semanal.index)

# Agrupar por mes y sumar los valores de inversión para cada mes
inversion_por_mes = serie_semanal.groupby(serie_semanal.index.to_period('M')).sum()

# Encontrar el mes con la mayor inversión total
mes_con_mas_inversion = inversion_por_mes.idxmax()

print("El mes con la mayor inversión total es:", mes_con_mas_inversion)
print("")

# Mostrar los cinco primeros valores de inversión agrupados por mes
inversion_por_mes_ordenadas = inversion_por_mes.sort_values(ascending=False)

print(inversion_por_mes_ordenadas.head(5))



El mes con la mayor inversión total es: 2020-08

fecha
2020-08    141710
2020-12    128370
2019-08     80200
2020-03     76770
2020-02     69320
Freq: M, dtype: int64


### Combinando datos

Combina el dataframe de ventas con las descripciones de los productos, y quédate sólo con los productos de helados de tarrina.

In [41]:
# Cargar los DataFrames de ventas de helados y nombres de productos
ventas_helados = pd.read_excel('./ventas_helados.xlsx')
nombres_productos = pd.read_csv('./nombres_productos.csv')

# Combinar los DataFrames
ventas_con_nombres = pd.merge(ventas_helados, nombres_productos, on='id_producto', how='left')

print(ventas_con_nombres)


          fecha  id_producto  ventas_unidades  precio  \
0    2019-01-07         4758             7953    4.82   
1    2019-01-07           17             7052    1.50   
2    2019-01-07         9134             6403    2.36   
3    2019-01-07         8689             4266    2.47   
4    2019-01-07          402             2932    2.89   
...         ...          ...              ...     ...   
1388 2020-12-28         5684              854    3.96   
1389 2020-12-28          101              758    3.31   
1390 2020-12-28         3636              401    2.68   
1391 2020-12-28         7118              344    2.15   
1392 2020-12-28         5019              250    2.42   

                                                 nombre  
0       NESTLE helado maxibon cookies caja 4 uds 360 gr  
1                    OREO Helado cono caja 4 uds 250 gr  
2      DIA Helado cono sabor vainilla caja 6 uds 408 gr  
3     NESTLE Helado cono kit kat classic caja 4 uds ...  
4     MILKA Helado cono c

In [42]:
# Filtrar solo los productos de helados de tarrina
productos_helados_tarrina = ventas_con_nombres[ventas_con_nombres['nombre'].str.contains('Tarrina')]

print(productos_helados_tarrina)


          fecha  id_producto  ventas_unidades  precio  \
5    2019-01-07         2033             1862    2.44   
8    2019-01-07          240             1015    2.30   
10   2019-01-07         5684              317    3.70   
11   2019-01-07          101              208    3.93   
13   2019-01-07         3636              160    2.60   
...         ...          ...              ...     ...   
1388 2020-12-28         5684              854    3.96   
1389 2020-12-28          101              758    3.31   
1390 2020-12-28         3636              401    2.68   
1391 2020-12-28         7118              344    2.15   
1392 2020-12-28         5019              250    2.42   

                                                 nombre  
5            CARTE D' OR Helado de yogur Tarrina 500 gr  
8              Helado Haagen Dazs Cookies Tarrina 400 g  
10                         DIA Helado Fresa Tarrina 1kg  
11    MAGNUM Helado White Chocolate & Cookies Tarrin...  
13      CARTE D' OR Helad

Agrega el dataset anterior en una serie semanal que agregue todos los productos mediante la suma de unidades y la media de los precios

In [43]:
# Calcular la suma de unidades y la media de precios por semana
serie_semanal_tarrina = productos_helados_tarrina.groupby(pd.Grouper(key='fecha', freq='W-MON'))\
    .agg({'ventas_unidades': 'sum', 'precio': 'mean'})

print("Serie semanal para productos de helados de tarrina con la suma de unidades y la media de precios:")
print(serie_semanal_tarrina)

Serie semanal para productos de helados de tarrina con la suma de unidades y la media de precios:
            ventas_unidades    precio
fecha                                
2019-01-07             3694  2.903333
2019-01-14             3052  2.846667
2019-01-21             2952  2.820000
2019-01-28             3165  2.890000
2019-02-04             3211  2.925000
...                     ...       ...
2020-11-30             4144  2.827143
2020-12-07             5336  2.765714
2020-12-14             5566  2.797143
2020-12-21             5853  2.777143
2020-12-28             6090  2.754286

[104 rows x 2 columns]


##### Combina todos los dataframes en uno solo (llamado dataset_conjunto), partiendo del dataset con los datos de helados de tarrina agregados por semana. Utiliza left joins   

Crea una variable binaria <i>navidad</i> que valga 1 en las semanas de nochebuena y nochevieja


In [44]:
ventas_helados = pd.read_excel("ventas_helados.xlsx")
nombres_productos = pd.read_csv("nombres_productos.csv")
inversiones_helados = pd.read_excel("inversiones_helados.xlsx")

# Realizar left joins
dataset_conjunto = pd.merge(ventas_helados, nombres_productos, on="id_producto", how="left")
dataset_conjunto = pd.merge(dataset_conjunto, inversiones_helados, on="fecha", how="left")

# Mostrar las primeras filas del dataset combinado
print('DATASET CONJUNTO \n')
print(dataset_conjunto.head())

DATASET CONJUNTO 

       fecha  id_producto  ventas_unidades  precio  \
0 2019-01-07         4758             7953    4.82   
1 2019-01-07           17             7052    1.50   
2 2019-01-07         9134             6403    2.36   
3 2019-01-07         8689             4266    2.47   
4 2019-01-07          402             2932    2.89   

                                              nombre  inversion_tv  \
0    NESTLE helado maxibon cookies caja 4 uds 360 gr          6100   
1                 OREO Helado cono caja 4 uds 250 gr          6100   
2   DIA Helado cono sabor vainilla caja 6 uds 408 gr          6100   
3  NESTLE Helado cono kit kat classic caja 4 uds ...          6100   
4  MILKA Helado cono chocolate y vainilla Caja 4 ...          6100   

   inversion_radio  inversion_youtube  inversion_facebook  
0                0               6100                   0  
1                0               6100                   0  
2                0               6100                  

### Jugando con expresiones regulares

¿Eres capaz de crear una variable de formato (peso en kilos) a partir del nombre del producto, utilizando expresiones regulares?

In [45]:
def extraer_peso(nombre_producto):
    # Buscar el patrón de peso en gramos
    patron = r"(\d+)\s*gr"
    resultado = re.search(patron, nombre_producto, re.IGNORECASE)
    if resultado:
        peso_gramos = int(resultado.group(1))
        # Convertir gramos a kilogramos
        peso_kilos = peso_gramos / 1000
        return peso_kilos
    else:
        return None

# Aplicar la función a cada nombre de producto en la columna "nombre"
dataset_conjunto["peso_kilos"] = dataset_conjunto["nombre"].apply(extraer_peso)

# Imprimir el DataFrame resultante
print(dataset_conjunto["peso_kilos"])

0       0.360
1       0.250
2       0.408
3       0.240
4       0.270
        ...  
1388      NaN
1389    0.300
1390    0.500
1391      NaN
1392      NaN
Name: peso_kilos, Length: 1393, dtype: float64
